In [28]:
! pip install -Uqq torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu118
! pip install -Uqq transformers datasets evaluate accelerate ujson
import torch
print(f"CUDA={torch.cuda.is_available()}")
! pip freeze

True
accelerate==0.33.0
aiohttp==3.9.5
aiosignal==1.3.1
asttokens==2.4.1
async-timeout==4.0.3
attrs==23.2.0
certifi==2024.7.4
charset-normalizer==3.3.2
comm==0.2.2
datasets==2.20.0
debugpy==1.8.2
decorator==5.1.1
dill==0.3.8
evaluate==0.4.2
exceptiongroup==1.2.2
executing==2.0.1
filelock==3.13.1
frozenlist==1.4.1
fsspec==2024.2.0
huggingface-hub==0.24.2
idna==3.7
ipykernel==6.29.5
ipython==8.26.0
jedi==0.19.1
Jinja2==3.1.3
jupyter_client==8.6.2
jupyter_core==5.7.2
MarkupSafe==2.1.5
matplotlib-inline==0.1.7
mpmath==1.3.0
multidict==6.0.5
multiprocess==0.70.16
nest-asyncio==1.6.0
networkx==3.2.1
numpy==1.26.3
nvidia-cublas-cu11==11.11.3.6
nvidia-cuda-cupti-cu11==11.8.87
nvidia-cuda-nvrtc-cu11==11.8.89
nvidia-cuda-runtime-cu11==11.8.89
nvidia-cudnn-cu11==9.1.0.70
nvidia-cufft-cu11==10.9.0.58
nvidia-curand-cu11==10.3.0.86
nvidia-cusolver-cu11==11.4.1.48
nvidia-cusparse-cu11==11.7.5.86
nvidia-nccl-cu11==2.20.5
nvidia-nvtx-cu11==11.8.86
packaging==24.1
pandas==2.2.2
parso==0.8.4
pexpect==4.9

# CAUSAL

In [2]:
from datasets import load_dataset
eli5 = load_dataset("eli5_category", split="train[:5000]")
eli5 = eli5.flatten().train_test_split(test_size=0.2)
eli5["train"][0]

/home/msid/fastai/venv/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


{'q_id': '7g8a3o',
 'title': 'Can you damage the inside of your body if you constantly eat hot food and drink hot beverages?',
 'selftext': '',
 'category': 'Biology',
 'subreddit': 'explainlikeimfive',
 'answers.a_id': ['dqh8im1', 'dqh9r3t'],
 'answers.text': ["If the temperature of the food is high enough to burn your skin outside the body it will do so inside as well. Otherwise food cooling to body temperature isn't really a strain on the body.",
  "Hot as in spicy food can cause distress in your GI tract, due to the chemicals irritating the tissues lining it. Otherwise, you should be fine as long as the food isn't hot enough to cause physical damage. If you experience persistent discomfort when eating certain foods, it might not be temperature that is the issue but some other form of irritation. A doctor would probably be able to assist you better."],
 'answers.score': [6, 3],
 'answers.text_urls': [[], []],
 'title_urls': ['url'],
 'selftext_urls': ['url']}

In [3]:
from transformers import AutoTokenizer
tokenizer = AutoTokenizer.from_pretrained("distilbert/distilgpt2")

In [4]:
def preprocess_function(examples):
    return tokenizer([" ".join(x) for x in examples["answers.text"]])

preprocess_function(eli5["train"][0])

{'input_ids': [[40, 277, 220, 220, 256, 289, 304, 220, 220, 256, 304, 285, 279, 304, 374, 257, 256, 334, 374, 304, 220, 220, 267, 277, 220, 220, 256, 289, 304, 220, 220, 277, 267, 267, 288, 220, 220, 1312, 264, 220, 220, 289, 1312, 308, 289, 220, 220, 304, 299, 267, 334, 308, 289, 220, 220, 256, 267, 220, 220, 275, 334, 374, 299, 220, 220, 331, 267, 334, 374, 220, 220, 264, 479, 1312, 299, 220, 220, 267, 334, 256, 264, 1312, 288, 304, 220, 220, 256, 289, 304, 220, 220, 275, 267, 288, 331, 220, 220, 1312, 256, 220, 220, 266, 1312, 300, 300, 220, 220, 288, 267, 220, 220, 264, 267, 220, 220, 1312, 299, 264, 1312, 288, 304, 220, 220, 257, 264, 220, 220, 266, 304, 300, 300, 764, 220, 220, 440, 256, 289, 304, 374, 266, 1312, 264, 304, 220, 220, 277, 267, 267, 288, 220, 220, 269, 267, 267, 300, 1312, 299, 308, 220, 220, 256, 267, 220, 220, 275, 267, 288, 331, 220, 220, 256, 304, 285, 279, 304, 374, 257, 256, 334, 374, 304, 220, 220, 1312, 264, 299, 705, 256, 220, 220, 374, 304, 257, 300, 300,

In [5]:
tokenized_eli5 = eli5.map(
    preprocess_function,
    batched=True,
    num_proc=4,
    remove_columns=eli5["train"].column_names,
)

Map (num_proc=4):   0%|          | 0/4000 [00:00<?, ? examples/s]Token indices sequence length is longer than the specified maximum sequence length for this model (4076 > 1024). Running this sequence through the model will result in indexing errors
Token indices sequence length is longer than the specified maximum sequence length for this model (1091 > 1024). Running this sequence through the model will result in indexing errors
Token indices sequence length is longer than the specified maximum sequence length for this model (1376 > 1024). Running this sequence through the model will result in indexing errors
Map (num_proc=4):   0%|          | 0/1000 [00:00<?, ? examples/s]Token indices sequence length is longer than the specified maximum sequence length for this model (1282 > 1024). Running this sequence through the model will result in indexing errors
Token indices sequence length is longer than the specified maximum sequence length for this model (1359 > 1024). Running this sequence

In [6]:
block_size = 128

def group_texts(examples):
    # Concatenate all texts.
    concatenated_examples = {k: sum(examples[k], []) for k in examples.keys()}
    total_length = len(concatenated_examples[list(examples.keys())[0]])
    # We drop the small remainder, we could add padding if the model supported it instead of this drop, you can
    # customize this part to your needs.
    if total_length >= block_size:
        total_length = (total_length // block_size) * block_size
    # Split by chunks of block_size.
    result = {
        k: [t[i : i + block_size] for i in range(0, total_length, block_size)]
        for k, t in concatenated_examples.items()
    }
    result["labels"] = result["input_ids"].copy()
    return result

In [7]:
lm_dataset = tokenized_eli5.map(group_texts, batched=True, num_proc=4)

Map (num_proc=4): 100%|██████████| 1000/1000 [00:00<00:00, 4463.56 examples/s]


In [8]:
from transformers import DataCollatorForLanguageModeling

tokenizer.pad_token = tokenizer.eos_token
data_collator = DataCollatorForLanguageModeling(tokenizer=tokenizer, mlm=False)

In [10]:
from transformers import AutoModelForCausalLM, TrainingArguments, Trainer

model = AutoModelForCausalLM.from_pretrained("distilbert/distilgpt2")
training_args = TrainingArguments(
    output_dir="causal",
    eval_strategy="epoch",
    learning_rate=2e-5,
    weight_decay=0.01,
)
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=lm_dataset["train"],
    eval_dataset=lm_dataset["test"],
    data_collator=data_collator,
)
trainer.train()
tokenizer.save_pretrained("causal")

Epoch,Training Loss,Validation Loss
1,3.952400,3.824386
2,3.855600,3.814416
3,3.819900,3.813229


TrainOutput(global_step=3882, training_loss=3.8807293804941074, metrics={'train_runtime': 470.4882, 'train_samples_per_second': 65.995, 'train_steps_per_second': 8.251, 'total_flos': 1014158013235200.0, 'train_loss': 3.8807293804941074, 'epoch': 3.0})

In [11]:
import math

eval_results = trainer.evaluate()
print(f"Perplexity: {math.exp(eval_results['eval_loss']):.2f}")

Perplexity: 45.30


In [25]:
from transformers import AutoModelForCausalLM,  AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("causal")
model = AutoModelForCausalLM.from_pretrained("causal/checkpoint-3882")

prompt = "Somatic hypermutation allows the immune system to"
inputs = tokenizer(prompt, return_tensors="pt").input_ids
outputs = model.generate(inputs, max_new_tokens=100, do_sample=True, top_k=50, top_p=0.95)
tokenizer.batch_decode(outputs, skip_special_tokens=True)

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


['Somatic hypermutation allows the immune system to be effectively weakened due to the different genetic predisposition that predisposes people to the disease. There is also genetic variation that can change your blood\'s sensitivity to certain types of virus. This is called "reverse transcription of a virus". It\'s called epigenetic transcription. When DNA is broken down, it\'s changed in the form of epigenetic variants (e.g. epigenetic mutations). This is where the genes come from (e.g. epigenetic variants). Some genes have a much higher']

# GANTARA

In [1]:
import ujson

data = []
with open("aldiwan.json") as file:
    for poet in ujson.load(file):
        for poem in poet['poems']:
            for i, half in enumerate(poem['halves']):
                if i == 0:
                    poem_concat = []
                elif i % 2 == 1:
                    poem_concat.append(' ∴ ')
                else:
                    poem_concat.append('\n')
                poem_concat.append(half)
            data.append(''.join(poem_concat))

print(len(data))
data[0]

38608


'هَل غادَرَ الشُعَراءُ مِن مُتَرَدَّمِ ∴ أَم هَل عَرَفتَ الدارَ بَعدَ تَوَهُّمِ\nيا دارَ عَبلَةَ بِالجَواءِ تَكَلَّمي ∴ وَعَمي صَباحاً دارَ عَبلَةَ وَاِسلَمي\nفَوَقَفتُ فيها ناقَتي وَكَأَنَّه ∴ فَدَنٌ لِأَقضِيَ حاجَةَ المُتَلَوِّمِ\nوَتَحُلُّ عَبلَةُ بِالجَواءِ وَأَهلُن ∴ بِالحَزنِ فَالصَمّانِ فَالمُتَثَلَّمِ\nحُيِّيتَ مِن طَلَلٍ تَقادَمَ عَهدُهُ ∴ أَقوى وَأَقفَرَ بَعدَ أُمِّ الهَيثَمِ\nحَلَّت بِأَرضِ الزائِرينَ فَأَصبَحَت ∴ عَسِراً عَلَيَّ طِلابُكِ اِبنَةَ مَخرَمِ\nعُلِّقتُها عَرَضاً وَأَقتُلُ قَومَه ∴ زَعماً لَعَمرُ أَبيكَ لَيسَ بِمَزعَمِ\nوَلَقَد نَزَلتِ فَلا تَظُنّي غَيرَهُ ∴ مِنّي بِمَنزِلَةِ المُحَبِّ المُكرَمِ\nكَيفَ المَزارُ وَقَد تَرَبَّعَ أَهلُه ∴ بِعُنَيزَتَينِ وَأَهلُنا بِالغَيلَمِ\nإِن كُنتِ أَزمَعتِ الفِراقَ فَإِنَّم ∴ زُمَّت رِكابُكُمُ بِلَيلٍ مُظلِمِ\nما راعَني إِلّا حَمولَةُ أَهلِه ∴ وَسطَ الدِيارِ تَسَفُّ حَبَّ الخِمخِمِ\nفيها اِثنَتانِ وَأَربَعونَ حَلوبَةً ∴ سوداً كَخافِيَةِ الغُرابِ الأَسحَمِ\nإِذ تَستَبيكَ بِذي غُروبٍ واضِحٍ ∴ عَذبٍ مُقَبَّلُهُ لَذيذِ المَطعَمِ\nوَ

In [2]:
from datasets import Dataset, DatasetDict
import random

data_list = [{'text': d}  for d in data][:10000]
train_test = Dataset.from_list(data_list).train_test_split(test_size=.5)
test_valid = train_test['test'].train_test_split(test_size=.5)
dataset = DatasetDict({
    'train': train_test['train'],
    'test': test_valid['train'],
    'valid': test_valid['test']})
print(dataset['train'][0])
dataset

/home/msid/fastai/venv/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


{'text': 'الصادعون صَفَا من لا يواجدهم ∴ والمُرْأبون بأذن الله ما شعَبوا'}


DatasetDict({
    train: Dataset({
        features: ['text'],
        num_rows: 5000
    })
    test: Dataset({
        features: ['text'],
        num_rows: 2500
    })
    valid: Dataset({
        features: ['text'],
        num_rows: 2500
    })
})

In [3]:
from transformers import AutoTokenizer
model_name = "aubmindlab/aragpt2-base"
tokenizer = AutoTokenizer.from_pretrained(model_name)

In [4]:
def preprocess_function(examples):
    return tokenizer([" ".join(x) for x in examples["text"]])

preprocess_function(dataset["train"][0])

{'input_ids': [[271], [153, 230], [311], [271], [277], [276], [262], [263], [224], [311], [153, 240], [279], [153, 240], [271], [224], [153, 231], [263], [224], [153, 230], [271], [224], [153, 236], [262], [271], [299], [277], [280], [153, 231], [224], [162, 234, 116], [224], [262], [271], [153, 230], [153, 231], [153, 241], [269], [153, 244], [349], [275], [262], [263], [224], [275], [349], [333], [263], [224], [271], [153, 230], [153, 230], [280], [224], [153, 231], [271], [224], [313], [276], [153, 240], [275], [262], [271]], 'attention_mask': [[1], [1, 1], [1], [1], [1], [1], [1], [1], [1], [1], [1, 1], [1], [1, 1], [1], [1], [1, 1], [1], [1], [1, 1], [1], [1], [1, 1], [1], [1], [1], [1], [1], [1, 1], [1], [1, 1, 1], [1], [1], [1], [1, 1], [1, 1], [1, 1], [1], [1, 1], [1], [1], [1], [1], [1], [1], [1], [1], [1], [1], [1], [1, 1], [1, 1], [1], [1], [1, 1], [1], [1], [1], [1], [1, 1], [1], [1], [1]]}

In [5]:
tokenized_dataset = dataset.map(
    preprocess_function,
    batched=True,
    num_proc=4,
    remove_columns=dataset["train"].column_names,
)

Map (num_proc=4): 100%|██████████| 2500/2500 [00:01<00:00, 1852.37 examples/s]


In [6]:
block_size = 128

def group_texts(examples):
    # Concatenate all texts.
    concatenated_examples = {k: sum(examples[k], []) for k in examples.keys()}
    total_length = len(concatenated_examples[list(examples.keys())[0]])
    # We drop the small remainder, we could add padding if the model supported it instead of this drop, you can
    # customize this part to your needs.
    if total_length >= block_size:
        total_length = (total_length // block_size) * block_size
    # Split by chunks of block_size.
    result = {
        k: [t[i : i + block_size] for i in range(0, total_length, block_size)]
        for k, t in concatenated_examples.items()
    }
    result["labels"] = result["input_ids"].copy()
    return result

lm_dataset = tokenized_dataset.map(group_texts, batched=True, num_proc=4)

Map (num_proc=4): 100%|██████████| 2500/2500 [00:04<00:00, 589.80 examples/s]


In [7]:
from transformers import DataCollatorForLanguageModeling

tokenizer.pad_token = tokenizer.eos_token
data_collator = DataCollatorForLanguageModeling(tokenizer=tokenizer, mlm=False)

In [8]:
from transformers import AutoModelForCausalLM, TrainingArguments, Trainer

model = AutoModelForCausalLM.from_pretrained(model_name)
training_args = TrainingArguments(
    output_dir="gantara",
    eval_strategy="epoch",
    learning_rate=2e-5,
    weight_decay=0.01,
)
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=lm_dataset["train"],
    eval_dataset=lm_dataset["test"],
    data_collator=data_collator,
)
trainer.train()
tokenizer.save_pretrained("gantara")

Epoch,Training Loss,Validation Loss
1,1.036300,0.978823
2,0.969600,0.929720
3,0.943600,0.915054


('gantara/tokenizer_config.json',
 'gantara/special_tokens_map.json',
 'gantara/vocab.json',
 'gantara/merges.txt',
 'gantara/added_tokens.json',
 'gantara/tokenizer.json')

In [9]:
from transformers import pipeline
from transformers import AutoTokenizer

prompt = "هل غادر الشعراء من متردم"
tokenizer = AutoTokenizer.from_pretrained("gantara")
generator = pipeline("text-generation",
                     model="gantara/checkpoint-20000", tokenizer=tokenizer)
generator(prompt)

OSError: gantara/checkpoint-20000 is not a local folder and is not a valid model identifier listed on 'https://huggingface.co/models'
If this is a private repository, make sure to pass a token having permission to this repo either by logging in with `huggingface-cli login` or by passing `token=<your_token>`